# Debug Notebook for GitHub Copilot Usage

This notebook demonstrates how to instantiate and invoke methods from your local exporter scripts. You can use it to debug the data extraction and adaptation processes for GitHub Copilot usage data.

In [1]:
# If your project structure is something like:
# PROJECT_ROOT/
#   |-- notebooks/debug_copilot_usage.ipynb
#   |-- src/
#        |-- extractor/
#             |-- github_exporter.py
#             |-- github_copilot_exporter.py
#        |-- common/
#             |-- common.py
#        |-- ...
# make sure to add the project root to sys.path if needed.

import sys
import os
project_root = os.path.abspath(os.path.join('..'))  # Adjust path if needed
if project_root not in sys.path:
    sys.path.append(project_root)

# Now we can import our local modules:


## Prepare a Configuration Dictionary
Below is a sample configuration dictionary. Adjust the values according to your project’s needs or how you manage configs (e.g., from a .ini or .env file).

In [2]:

from src.extractor.github_copilot_exporter import GithubCopilotExporter
import pandas as pd

pd.options.mode.chained_assignment = None

config = {
    "GITHUB": {
        "github_url": "https://api.github.com",
        "github_org": "gologic-partner",     # Replace with your organization
        "github_token": "",  # Replace with your token
    },
}

# Instantiate the Exporter
copilot_exporter = GithubCopilotExporter()
copilot_exporter.initialize_data(config)

## Call the Copilot Usage Endpoint
We’ll do a simple request to GitHub’s Copilot usage endpoint. Adjust the query parameters (e.g., since, until) as needed.

For reference, see: [Copilot Usage Docs](https://docs.github.com/en/rest/copilot/copilot-usage?apiVersion=2022-11-28).

In [3]:
endpoint = f"orgs/{copilot_exporter.github_org}/copilot/metrics"
# If you want to pass query parameters, you could do something like:
# endpoint = f"orgs/{copilot_exporter.github_org}/copilot/usage?since=2023-01-01&until=2023-01-31"

usage_data = copilot_exporter.execute_simple_request(endpoint)

print("Raw Usage Data (truncated):")
print(usage_data[:1])  # Display only the first element if it’s a list

Raw Usage Data (truncated):
[{'date': '2025-01-02', 'copilot_ide_chat': {'total_engaged_users': 0}, 'total_active_users': 2, 'copilot_dotcom_chat': {'total_engaged_users': 0}, 'total_engaged_users': 1, 'copilot_dotcom_pull_requests': {'total_engaged_users': 0}, 'copilot_ide_code_completions': {'editors': [{'name': 'vscode', 'models': [{'name': 'default', 'languages': [{'name': 'yaml', 'total_engaged_users': 1, 'total_code_acceptances': 4, 'total_code_suggestions': 67, 'total_code_lines_accepted': 3, 'total_code_lines_suggested': 187}], 'is_custom_model': False, 'total_engaged_users': 1}], 'total_engaged_users': 1}, {'name': 'JetBrains', 'models': [{'name': 'default', 'languages': [{'name': 'ini', 'total_engaged_users': 0, 'total_code_acceptances': 0, 'total_code_suggestions': 1, 'total_code_lines_accepted': 0, 'total_code_lines_suggested': 1}], 'is_custom_model': False}], 'total_engaged_users': 0}], 'languages': [{'name': 'yaml', 'total_engaged_users': 1}, {'name': 'ini', 'total_engage

## Adapt the Usage Data
Here we call the `adapt_usage` (which you can define in your exporter module following the same pattern as other `adapt_*` methods) to structure the data into a DataFrame or DataFrames.

In [4]:
# In case you already have a method like 'adapt_usage' in your GithubCopilotExporter
# you can call it directly. Otherwise, define it similarly to how you have adapt_pulls, etc.

adapted_usage = copilot_exporter.adapt_usage(usage_data)

# The result might be a dict of DataFrames:
df_usage = adapted_usage["df_usage"]
df_usage_breakdown = adapted_usage["df_usage_breakdown"]

print("DataFrame: df_usage")
display(df_usage.head())

print("DataFrame: df_usage_breakdown")
display(df_usage_breakdown.head())

AttributeError: 'GithubCopilotExporter' object has no attribute 'adapt_usage'

In [5]:
# Cell 2 - Code
# First get the teams
teams = copilot_exporter.extract_teams()
print("\nTeams found:")
for team in teams:
    print(f"- {team['name']} (slug: {team['slug']})")


Teams found:
- Équipe développement 1 (slug: equipe-developpement-1)
- Équipe GitHub (slug: equipe-github)
- go-squad (slug: go-squad)


c:\Users\SébastienBernard\Documents\GitHub\devops-metrics\.venv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
# Cell 3 - Code
# Get metrics for each team
metrics_per_team = copilot_exporter.extract_metrics_per_team(teams)
adapted_metrics_per_team = copilot_exporter.adapt_metrics_chat_team(metrics_per_team)

print("\nMetrics per team:")
for team_slug, metrics in metrics_per_team.items():
    print(f"\nTeam: {team_slug}")
    print(f"Number of data points: {len(metrics)}")
    if len(metrics) > 0:
        print("Sample metrics:")
        print(metrics[0])

c:\Users\SébastienBernard\Documents\GitHub\devops-metrics\.venv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\SébastienBernard\Documents\GitHub\devops-metrics\.venv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



Metrics per team:

Team: equipe-developpement-1
Number of data points: 1
Sample metrics:
{'date': '2025-01-28', 'copilot_ide_chat': {'editors': [{'name': 'vscode', 'models': [{'name': 'default', 'total_chats': 20, 'is_custom_model': False, 'total_engaged_users': 1, 'total_chat_copy_events': 0, 'total_chat_insertion_events': 0}], 'total_engaged_users': 1}, {'name': 'JetBrains', 'models': [{'name': 'default', 'total_chats': 16, 'is_custom_model': False, 'total_engaged_users': 3, 'total_chat_copy_events': 3, 'total_chat_insertion_events': 3}], 'total_engaged_users': 3}], 'total_engaged_users': 3}, 'total_active_users': 5, 'copilot_dotcom_chat': {'total_engaged_users': 0}, 'total_engaged_users': 5, 'copilot_dotcom_pull_requests': {'total_engaged_users': 0}, 'copilot_ide_code_completions': {'editors': [{'name': 'JetBrains', 'models': [{'name': 'default', 'languages': [{'name': 'csharp', 'total_engaged_users': 1, 'total_code_acceptances': 10, 'total_code_suggestions': 28, 'total_code_lines_

c:\Users\SébastienBernard\Documents\GitHub\devops-metrics\.venv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
adapted_metrics_per_team

[                     team        date editor_name model_name  \
 0  equipe-developpement-1  2025-01-28      vscode    default   
 1  equipe-developpement-1  2025-01-28   JetBrains    default   
 
    total_engaged_users  total_chat  total_chat_insertion_events  \
 0                    1          20                            0   
 1                    3          16                            3   
 
    total_chat_copy_events  
 0                       0  
 1                       3  ,
             team        date editor_name model_name  total_engaged_users  \
 0  equipe-github  2025-01-28   JetBrains    default                    3   
 1  equipe-github  2025-01-28      vscode    default                    2   
 
    total_chat  total_chat_insertion_events  total_chat_copy_events  
 0          16                            6                      15  
 1          33                            0                       6  ,
 Empty DataFrame
 Columns: []
 Index: []]

In [6]:
global_metrics = copilot_exporter.extract_metrics_global()

c:\Users\SébastienBernard\Documents\GitHub\devops-metrics\.venv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
metrics_ide = copilot_exporter.adapt_metrics_chat_global(global_metrics)
metrics_ide


,date,editor_name,model_name,total_engaged_users,total_chat,total_chat_insertion_events,total_chat_copy_events
0,2025-01-03,vscode,default,1,18,0,0
1,2025-01-05,JetBrains,default,1,8,0,0
2,2025-01-06,vscode,default,2,21,0,1
3,2025-01-06,JetBrains,default,2,8,2,2
4,2025-01-07,JetBrains,default,5,81,11,11
5,2025-01-07,vscode,default,4,29,0,0
6,2025-01-08,JetBrains,default,5,76,13,18
7,2025-01-08,vscode,default,3,21,0,4
8,2025-01-09,JetBrains,default,5,48,11,8
9,2025-01-09,vscode,default,3,52,0,5


In [10]:
metrics_completition = copilot_exporter.adapt_metrics_completitions_global(global_metrics)
metrics_completition

,date,editor_name,model_name,language,total_code_lines_suggested,total_code_lines_accepted,total_engaged_users
0,2025-01-02,vscode,default,yaml,3,0,1
1,2025-01-02,JetBrains,default,ini,0,0,0
2,2025-01-03,vscode,default,yaml,1,0,1
3,2025-01-03,Neovim,default,terraform,34,0,1
4,2025-01-04,VisualStudio,default,csharp,4,0,1
...,...,...,...,...,...,...,...
58,2025-01-27,vscode,default,unknown,0,0,1
59,2025-01-27,Neovim,default,terraform,0,0,0
60,2025-01-28,JetBrains,default,php,0,0,4
61,2025-01-28,Neovim,default,unknown,1,0,1


## Conclusion
You can now inspect these DataFrames, observe schema changes, and debug any issues with your transformation logic or with the GitHub API data itself. Use additional cells as needed to explore data, handle errors, etc.